In [ ]:
# ── 0) Install dependencies ────────────────────────────────────────────
!pip install --quiet pandas requests tqdm

# ── 1) Load existing files ────────────────────────────────────────────
import pandas as pd
import requests
import time
import random
import urllib.parse
from tqdm import tqdm

# Read your CSV and cookie
df = pd.read_csv("base.csv")
cookie = open("cookie.txt", "r", encoding="latin-1").read().strip()

# Ensure the 'const' column exists
if "const" not in df.columns:
    df.insert(0, "const", "")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/json",
    "cookie": cookie
}

# ── 2) OMDb API key ─────────────────────────────────────────────────────
OMDB_KEY = input("▶ Enter your free OMDb API key: ").strip()

# ── 3) Function to look up IMDb IDs via OMDb ────────────────────────────
def find_imdb_id(title: str, year: int) -> str | None:
    q = urllib.parse.quote(title)
    # Exact match (title + year)
    url1 = f"https://www.omdbapi.com/?t={q}&y={year}&apikey={OMDB_KEY}"
    r1 = requests.get(url1, timeout=10).json()
    if r1.get("Response") == "True":
        return r1.get("imdbID")
    # Broad search, then filter by year
    url2 = f"https://www.omdbapi.com/?s={q}&apikey={OMDB_KEY}"
    r2 = requests.get(url2, timeout=10).json()
    if r2.get("Response") == "True":
        for hit in r2["Search"]:
            if str(year) in hit.get("Year", ""):
                return hit["imdbID"]
        # Fallback to first result
        return r2["Search"][0]["imdbID"]
    return None

# ── 4) Fill in missing IMDb IDs ─────────────────────────────────────────
print("\n🔍 Looking up IMDb IDs via OMDb…")
found = 0
for i, row in tqdm(df.iterrows(), total=len(df)):
    if isinstance(row["const"], str) and row["const"].startswith("tt"):
        continue
    imdb_id = find_imdb_id(row["Title"], int(row["Year"]))
    if imdb_id:
        df.at[i, "const"] = imdb_id
        found += 1
    else:
        print(f"❓ Not found: {row['Title']} ({row['Year']})")
    time.sleep(random.uniform(0.2, 0.4))
print(f"✅ Found {found}/{len(df)} IMDb IDs")

# Save intermediate CSV
df.to_csv("imdb_ratings_ready.csv", index=False)
print("→ imdb_ratings_ready.csv created")

# ── 5) Function to submit ratings via IMDb GraphQL ────────────────────
def rate_imdb(tt: str, rating: int):
    payload = {
        "query": """
            mutation UpdateTitleRating($rating:Int!,$titleId:ID!){
              rateTitle(input:{rating:$rating,titleId:$titleId}){rating{value}}
            }""",
        "operationName": "UpdateTitleRating",
        "variables": {"rating": rating, "titleId": tt}
    }
    response = requests.post(
        "https://api.graphql.imdb.com/",
        headers=headers,
        json=payload,
        timeout=10
    )
    if response.status_code == 429:
        raise RuntimeError("IMDb rate-limited; please wait a bit…")
    if response.status_code != 200:
        raise RuntimeError(f"HTTP {response.status_code}: {response.text[:100]}")
    if response.json().get("errors"):
        raise RuntimeError(response.json()["errors"][0]["message"])

# ── 6) Upload all ratings ───────────────────────────────────────────────
print("\n⭐ Uploading ratings to IMDb…")
ok, ko = 0, 0
for _, row in tqdm(df.iterrows(), total=len(df)):
    tt, rating = row["const"], int(row["your rating"])
    if not str(tt).startswith("tt"):
        ko += 1
        continue
    try:
        rate_imdb(tt, rating)
        ok += 1
        time.sleep(random.uniform(0.3, 0.6))
    except Exception as e:
        print(f"⚠️ {tt} – {e}")
        ko += 1

print(f"\n🎉 Done: {ok} successes, {ko} failures")